<a href="https://colab.research.google.com/github/w95wayne10/computer_vision/blob/master/summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
#from google.colab.patches import cv2_imshow

In [ ]:
def Convolu(img, pattern):
  n,m = pattern.shape
  padding_img = np.zeros((img.shape[0]+n-1,img.shape[1]+m-1))
  n_size = (n-1)//2
  m_size = (m-1)//2
  padding_img[n_size:-n_size,m_size:-m_size]=img
  padding_img_to1 = padding_img//255
  convolu_img_to1 = np.zeros(img.shape)
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      convolu_img_to1[i,j] = np.sum(padding_img_to1[i:i+n,j:j+m]*pattern)
  return convolu_img_to1

octogonal_kernel = np.array([[0,1,1,1,0],
                             [1,1,1,1,1],
                             [1,1,1,1,1],
                             [1,1,1,1,1],
                             [0,1,1,1,0]])
J = np.array([[0,0,0],[1,1,0],[0,1,0]])
K = np.array([[0,1,1],[0,0,1],[0,0,0]])


pattern_sum = np.sum(octogonal_kernel)

def Dilation(convolu_img):
  return (convolu_img>0.5)*255

def Erosion(convolu_img):
  return (convolu_img>pattern_sum-0.5)*255

In [ ]:
# 讀檔
lena_gray = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

In [ ]:
# HW1
# 變更順序達到旋轉鏡射等效果
upside_down_lena = lena_gray[::-1, :]
rightside_left_lena = lena_gray[:, ::-1]
diagonally_flip_lena = lena_gray.T

cv2.imwrite('upside_down_lena.bmp', upside_down_lena)
cv2.imwrite('rightside_left_lena.bmp', rightside_left_lena)
cv2.imwrite('diagonally_flip_lena.bmp', diagonally_flip_lena)

In [ ]:
# 二元化
binarize_lena = lena_gray//128*255
cv2.imwrite('binarize_lena.bmp', binarize_lena)


# 分群
IDlist = []
IDlist.append(0)
IDtable = np.zeros((512,512),dtype=int)
ID = 1
for r, row in enumerate(binarize_lena):
  for c, pix in enumerate(row):
    if pix==0:
      continue
    if r:
      if c and IDtable[r-1,c-1]:
        IDtable[r,c] = IDtable[r-1,c-1]
      elif IDtable[r-1,c]:
        if IDtable[r,c]:
          IDlist[IDtable[r-1,c]]["neighbor"].add(IDtable[r,c])
          IDlist[IDtable[r,c]]["neighbor"].add(IDtable[r-1,c])
        else:
          IDtable[r,c] = IDtable[r-1,c]
      elif c<511 and IDtable[r-1,c+1]:
        if IDtable[r,c]:
          IDlist[IDtable[r-1,c+1]]["neighbor"].add(IDtable[r,c])
          IDlist[IDtable[r,c]]["neighbor"].add(IDtable[r-1,c+1])
        else:
          IDtable[r,c] = IDtable[r-1,c+1]
    if c and IDtable[r,c-1]:
      if IDtable[r,c]:
        IDlist[IDtable[r,c-1]]["neighbor"].add(IDtable[r,c])
        IDlist[IDtable[r,c]]["neighbor"].add(IDtable[r,c-1])
      else:
        IDtable[r,c] = IDtable[r,c-1]
    if not IDtable[r,c]:
      IDtable[r,c] = ID
      ID+=1
      IDlist.append({"up":512,"down":0,"left":512,"right":0,"count":0,"sum":[0,0],"neighbor":set()})
    IDlist[IDtable[r,c]]["up"] = min(IDlist[IDtable[r,c]]["up"],r)
    IDlist[IDtable[r,c]]["down"] = max(IDlist[IDtable[r,c]]["down"],r)
    IDlist[IDtable[r,c]]["left"] = min(IDlist[IDtable[r,c]]["left"],c)
    IDlist[IDtable[r,c]]["right"] = max(IDlist[IDtable[r,c]]["right"],c)
    IDlist[IDtable[r,c]]["count"] += 1
    IDlist[IDtable[r,c]]["sum"][0] += r
    IDlist[IDtable[r,c]]["sum"][1] += c
for i in range(len(IDlist)-1,1,-1):
  if not IDlist[i]['neighbor']:
    continue
  belong = min(IDlist[i]['neighbor'])
  if belong == i:
    continue
  for j in IDlist[i]['neighbor']:
    if i==j:
      continue
    IDlist[j]['neighbor'].add(belong)
    if i in IDlist[j]['neighbor']:
      IDlist[j]['neighbor'].remove(i)
  IDlist[belong]["up"] = min(IDlist[belong]["up"],IDlist[i]["up"])
  IDlist[belong]["down"] = max(IDlist[belong]["down"],IDlist[i]["down"])
  IDlist[belong]["left"] = min(IDlist[belong]["left"],IDlist[i]["left"])
  IDlist[belong]["right"] = max(IDlist[belong]["right"],IDlist[i]["right"])
  IDlist[belong]["count"] += IDlist[i]["count"]
  IDlist[belong]["sum"][0] += IDlist[i]["sum"][0]
  IDlist[belong]["sum"][1] += IDlist[i]["sum"][1]
  IDlist.pop(i)
for i in range(len(IDlist)-1,1,-1):
  if IDlist[i]["count"]<500:
    IDlist.pop(i)
image = cv2.cvtColor(binarize_lena, cv2.COLOR_GRAY2BGR)
for grp in IDlist:
  if grp == 0:
    continue
  image = cv2.circle(image, 
          (int(np.round(grp["sum"][1]/grp["count"])), int(np.round(grp["sum"][0]/grp["count"]))), 
          radius=0, 
          color=(0, 0, 255), 
          thickness=10)
  image = cv2.rectangle(image, (grp["left"],grp["up"]), (grp["right"],grp["down"]), (255,0, 0), 2)
cv2.imwrite('group_of_lena.bmp', image)

In [ ]:
divid_3_lena_gray = np.uint8(np.rint(lena_gray/3))
cv2.imwrite('divid_3_lena_gray.bmp', divid_3_lena_gray)

In [ ]:
# 灰階圖的色階統計

def gray_level_value_count(gray_img):
  value_count = [0]*256
  for row in gray_img:
    for pix in row:
      value_count[pix]+=1
  return value_count

def gray_level_value_count_chart(gray_img, out_fig_name):
  value_count = gray_level_value_count(gray_img)
  plt.figure(dpi=500)
  plt.bar(list(range(256)),value_count,1)
  plt.xlabel("gray level value")
  plt.xlim(0,255)
  plt.ylabel("count")
  plt.savefig(out_fig_name)
  return

gray_level_value_count_chart(lena_gray, 'value_count_lena.png')
gray_level_value_count_chart(divid_3_lena_gray, 'value_count_divid_3_lena_gray.png')

In [ ]:
def gray_hist_equalization(gray_img):
  value_cumsum = []
  minlevel, maxlevel, sum_value = -1, -1, 0
  novalue = True
  for level, count in enumerate(gray_level_value_count(gray_img)):
    sum_value += count
    value_cumsum.append(sum_value)
    if novalue and count>0:
      novalue=False
      minlevel = level
    if count>0:
      maxlevel = level

  new_value_count = [0]*256
  transfor_list = [0]*256
  Denominator = value_cumsum[maxlevel]-value_cumsum[minlevel]
  Numerator = lambda x: value_cumsum[x]-value_cumsum[minlevel]
  for level in range(minlevel, maxlevel+1):
    new_level = int(np.round(Numerator(level)*255/Denominator))
    transfor_list[level] = new_level
    new_value_count[new_level] += value_count[level]

  hist_equalization_lena_gray = divid_3_lena_gray.copy()
  for r,row in enumerate(hist_equalization_lena_gray):
    for c,_ in enumerate(row):
      hist_equalization_lena_gray[r,c] = transfor_list[hist_equalization_lena_gray[r,c]]
  cv2.imwrite('hist_equalization_lena_gray.bmp', hist_equalization_lena_gray)

plt.figure(dpi=500)
plt.bar(list(range(256)),new_value_count,1)
plt.xlabel("gray level value")
plt.xlim(0,255)
plt.ylabel("count")
plt.savefig('value_count_hist_equalization_lena_gray.png')

In [ ]:
convolu_binarize_lena = Convolu(binarize_lena, octogonal_kernel)

Dilation_binarize_lena = Dilation(convolu_binarize_lena)
Erosion_binarize_lena = Erosion(convolu_binarize_lena)
convolu_Erosion_binarize_lena = Convolu(Erosion_binarize_lena, octogonal_kernel)
Opening_binarize_lena = Dilation(convolu_Erosion_binarize_lena)
convolu_Dilation_binarize_lena = Convolu(Dilation_binarize_lena, octogonal_kernel)
Closing_binarize_lena = Erosion(convolu_Dilation_binarize_lena)
Jconvolu_binarize_lena = Convolu(binarize_lena, J)
pattern_sum = np.sum(J)
JErosion = Erosion(Jconvolu_binarize_lena)
Kconvolu_binarize_lena = Convolu(255-binarize_lena, K)
pattern_sum = np.sum(K)
KErosion = Erosion(Kconvolu_binarize_lena)
HitandMiss_binarize_lena = ((JErosion > 128)&(KErosion > 128))*255

cv2.imwrite('Dilation_binarize_lena.bmp', Dilation_binarize_lena)
cv2.imwrite('Erosion_binarize_lena.bmp', Erosion_binarize_lena)
cv2.imwrite('Opening_binarize_lena.bmp', Opening_binarize_lena)
cv2.imwrite('Closing_binarize_lena.bmp', Closing_binarize_lena)
cv2.imwrite('HitandMiss_binarize_lena.bmp', HitandMiss_binarize_lena)